In [ ]:
#Create a python class
import csv
import pandas as pd
import time
from openserver import OpenServer

class MonteCarloSimulation:
    
    OS = OpenServer()
    def __init__(self,app_names, input_links, output_links):
        self._app_names = app_names
        self._input_links = input_links
        self._output_links = output_links
       
    #Connect to or disconnect from openserver
    def control_openserver(self, condition = True):
        if condition:
            self.OS.connect()
            
        elif condition == False:
            self.OS.disconnect()
     
    #Open IPM or close it
    def launch_ipm(self, action = True):
#         self.control_openserver(True)
        for each_name in self._app_names:
            if action:
                each_name = each_name + '.START()'
                
            elif action == False:
                each_name = each_name + '.SHUTDOWN'
                
            MonteCarloSimulation.OS.DoCmd(each_name)
        
    #Open each ipm file and convert excel to dataframe        
    def open_ipm_files(self):
#       self.control_openserver(True)
        for each_file in self._input_links:
            if each_file["name"] in ['PROSPER', 'MBAL', 'GAP']:
                file_to_open = f'{each_file["name"]}.OPENFILE(\"' + f'C:\\Users\\ftkw\\Downloads\\OKANAG\\{each_file["link"]}\")'
                MonteCarloSimulation.OS.DoCmd(file_to_open)
            elif each_file["name"] == 'EXCEL':
                excel_link = f'C:\\Users\\ftkw\\Downloads\\OKANAG\\{each_file["link"]}'
                mc_df = pd.read_excel(excel_link, sheet_name = each_file["sheet_name"])
                
        return mc_df
                
    
    def kill_openserver(self):
        import psutil
        process_names = ['gap.exe', 'mbal.exe', 'prosper.exe']
        for proc in psutil.process_iter():
            if proc.name() in process_names:
                proc.kill()
                
    #Do the simulation here            
    def run_monte_carlo(self, input_string, output_string):
        mc_df = self.open_ipm_files()
        excel_link = [x['link'] for x in self._input_links if '.xlsx' in x["link"]][0]
        excel_link = f'C:\\Users\\ftkw\\Downloads\\OKANAG\\{excel_link}'
        for index, row in mc_df.iterrows():
            if pd.isna(mc_df.at[index, 'Cum Cond Prod']):
                    print(index)
                    for each_string in input_string:
                        MonteCarloSimulation.OS.DoSet(each_string["op_str"], row[each_string["prop"]])

    #                  Save MBAL and PROSPER files   
                    for each_file in self._input_links:
                        if (each_file["name"] in ["MBAL", "PROSPER"]):
                            file_to_save = f'{each_file["name"]}.SAVEFILE(\"' + f'C:\\Users\\ftkw\\Downloads\\OKANAG\\{each_file["link"]}\")'
                            MonteCarloSimulation.OS.DoCmd(file_to_save)
                            time.sleep(0.01)
                    tpd_file = [x["link"] for x in self._input_links if x["name"] == "TPD"]
                    tpd_file = tpd_file[0]
                    file_for_tpd = f"C:\\Users\\ftkw\\Downloads\\OKANAG\\{tpd_file}"
                    MonteCarloSimulation.OS.DoCmd('PROSPER.ANL.VLP.CALC') #calculate vlp
                    MonteCarloSimulation.OS.DoSet('PROSPER.ANL.VLP.EXP.Sel',8) #select petroleum experts export format
                    MonteCarloSimulation.OS.DoSet('PROSPER.ANL.VLP.EXP.File',file_for_tpd)
                    MonteCarloSimulation.OS.DoCmd('PROSPER.ANL.VLP.Export') #export the tpd file

                    #import vlp into GAP
                    my_vlp = [x for x in output_string if x["prop"] == "vlp_import"]
                    vlp_import = f'GAP.VLPIMPORT({my_vlp[0]["op_str"]}, \"{file_for_tpd}\")'
                    MonteCarloSimulation.OS.DoCmd(vlp_import)

                    #export prosper ipr into GAP
                    ipr_string = [x["op_str"] for x in output_string if x["prop"] == "ipr_transfer"]
                    ipr_string = ipr_string[0] + ',0,0)'
                    MonteCarloSimulation.OS.DoCmd(ipr_string)


                    ##run solver in GAP##
                    MonteCarloSimulation.OS.DoCmd("GAP.SOLVENETWORK(1,MOD[{PROD}])")
                    for each_file in output_string:
                        if "Solver" in each_file["op_str"]:
                                mc_df.at[index, each_file["prop"]] = MonteCarloSimulation.OS.DoGet(each_file["op_str"])

                        ##run gap in prediction mode and retrieve cum gas and cum oil at end of prediction##
                    
                    MonteCarloSimulation.OS.DoCmd('GAP.PREDINIT(0,0)') #Initialises prediction and returns the 
                    timesteps = 31 #total number of prediction steps. 
                        #get timesteps by running a sample pred in GAP manually
                        #ensure the end date you put into pred run is one timestep after the date you want to extract data for
                        #e.g. I want to extract data for 1/1/2055, I put in 1/1/2056 as my pred end date
                        #now check the the last step# at the end of the pred run. this is your timesteps

                    for i in range(1,timesteps+1,1):
                        MonteCarloSimulation.OS.DoCmd('GAP.PREDDOSTEP(1,0)') #loop through the timesteps in increment of 1 timesteps

                    for each_file in output_string:
                        if "PREDRES" in each_file["op_str"]:
                            mc_df.at[index, each_file["prop"]] = MonteCarloSimulation.OS.DoGet(each_file["op_str"])
    
                    MonteCarloSimulation.OS.DoCmd('GAP.PREDEND(1,1,0)') #Finalises the Prediction run in GAP restoring initial values in the model
            
                    file_to_save = [x["link"] for x in self._input_links if x["name"] == "GAP"]
                    file_to_save = file_to_save[0]
                    file_to_save = f'GAP.SAVEFILE(\"' + f'C:\\Users\\ftkw\\Downloads\\OKANAG\\{file_to_save}\")'
                    MonteCarloSimulation.OS.DoCmd(file_to_save)
                    time.sleep(0.07) #introduce a small pause inbetween runs to avoid openserver license timeout
                    mc_df.to_excel(excel_link, sheet_name = "df")
                    continue
                    
        print('..saving...')
#         mc_df.to_csv('C:\\Users\\ftkw\\Downloads\\OKANAG\\mc_df_completedruns.csv')
        print(mc_df)
        print('saved')

       
        
    def repeat_mcruns(self,input_string,output_string):
        try:
            self.control_openserver(True)
            self.launch_ipm(True)
            self.open_ipm_files()
            self.run_monte_carlo(input_string, output_string)
        
        except ValueError:
            print("Looping back to reconnect")
            self.kill_openserver()
            self.repeat_mcruns(input_string, output_string)
            
    
            
 
            




In [ ]:
with open("data_files//app_names.csv", "r") as f:
    app_name = csv.DictReader(f)
    app_name = [x["name"] for x in app_name]
    
with open("data_files//file_links.csv", "r") as f:
    file_link = csv.DictReader(f)
    file_link = list(file_link)
                 
with open("data_files//output_string_files.csv", "r") as f:
    output_string_file = csv.DictReader(f)
    output_string_file = list(output_string_file)
     
with open("data_files//input_string_files.csv") as f:
    input_string_file = csv.DictReader(f)
    input_string_file = list(input_string_file)
    
        

In [ ]:
#Run instances

monte_carlo_runs = MonteCarloSimulation(app_name,file_link, output_string_file)
monte_carlo_runs.repeat_mcruns(input_string_file, output_string_file)



In [ ]:
monte_carlo_runs.kill_openserver()
monte_carlo_runs.control_openserver(False)